In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [20]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        # Outer Tag Object
        price = soup.find("span", attrs={'class':'a-price aok-align-center'})
        
        # Delete double value
        price_value = price.find('span', attrs={'class':'a-offscreen'}).text

    except AttributeError:
        price_value = ""

    return price_value

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find('span', attrs={'id':'acrCustomerReviewText'}).string.strip()
    
    except AttributeError:
        rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find('span', attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        # Outer Tag Object
        available = soup.find('div', attrs={'id':'availability'})
        
        # Inner NavigatableString Object
        available = available.find('span', attrs={'class': 'a-size-medium a-color-success'}).text.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [23]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+5&crid=3BB56JVU8Q2HB&sprefix=playstation+5%2Caps%2C558&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all('a', attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_project-web-scrapping.csv", header=True, index=False)

In [24]:
amazon_df.head()

,title,price,rating,reviews,availability
0,PlayStation PS5 Console – God of War Ragnarök ...,$509.00,"8,614 ratings","8,614 ratings",In Stock
1,PlayStation 5 Console CFI-1215A01X,$499.00,"3,278 ratings","3,278 ratings",In Stock
2,PlayStation 5 Console CFI-1102A,,"7,306 ratings","7,306 ratings",
3,PlayStation 5 Pulse 3D Wireless Headset,,"19,562 ratings","19,562 ratings",In Stock
4,PlayStation 5 Digital Edition,,"7,200 ratings","7,200 ratings",
